<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/basic_examples/template_for_channel_network_and_basemap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template for making a nice image of a channel network

Last updated by Simon M Mudd on 25/11/2024

This is a template notebook for picking a place and making a nice image of the channel network, alongside some two basemaps.

If you use this software for your research, please cite https://doi.org/10.5281/zenodo.3245040

## If you are on colab

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this.
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **First get data** section.

First we install `lsdtopotools`. The first line downloads the package and the second installs it. The `/dev/null` stuff is just to stop the notebook printing a bunch of text to screen.  

In [1]:
!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.9.

In [2]:
!lsdtt-basic-metrics -v

|| Welcome to the LSDTopoTools basic metrics tool!     ||
|| This program has a number of options for calculating||
|| simple landscape metrics.                           ||
|| This program was developed by Simon M. Mudd         ||
||  at the University of Edinburgh                     ||
|| If you use these routines please cite:              ||
|| http://doi.org/10.5281/zenodo.4577879               ||
|| If you use the roughness routine please cite:       ||
|| https://www.doi.org/10.5194/esurf-3-483-2015        ||
|| Documentation can be found at:                      ||
|| https://lsdtopotools.github.io/LSDTT_documentation/ ||
|| This is LSDTopoTools2 version                       ||
|| 0.9
|| If the version number has a d at the end it is a    ||
||  development version.                               ||
|| You have called an LSDTopoTools program.            ||
|| Prepare to explore topographic data!                ||
|| You can find some examples of usage here:           ||
|| http

Now we install `lsdviztools`:

In [3]:
!pip install lsdviztools  &> /dev/null

This template needs `lsdviztools` version 0.4.14 or higher, so test:

In [4]:
import lsdviztools
lsdviztools.__version__

'0.4.14'

## Now get some data

We need to get some data to download.

For this example we will work on the toe of the boot in Calabria, Italy.

We are going to download data using the opentopography scraper that is included with `lsdviztools`. You will need to get an opentopography.org account and copy in your API key.

You can sign up to an opentopography.org account here: https://portal.opentopography.org/myopentopo

In [5]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

Dataset_prefix = "RC"
source_name = "COP30"

my_DEM = bmt.ot_scraper(source = source_name,
                        lower_left_coordinates = [40.95399473322263, 69.65505930582663],
                        upper_right_coordinates = [42.593615267006946, 71.75292435647813],
                        prefix = Dataset_prefix,
                        api_key_file = your_OT_api_key_file)
my_DEM.print_parameters()
my_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_prefix+"_"+source_name+".tif"
gio.convert4lsdtt(DataDirectory,Fname)

I am reading you OT API key from the file my_OT_api_key.txt
Your api key starts with: 9515
I am taking your coordinates from the lower left list
I am taking your coordinates from the upper right list
I am reading you OT API key from the file my_OT_api_key.txt
The grid spacing for your DEM will be:30
The source is: COP30
The west longitude is: 69.65505930582663
The east longitude is: 71.75292435647813
The south latitude is: 42.593615267006946
The north latitude is: 40.95399473322263
The path is: ./
The prefix is: RC
The grid spacing is: 30
I am going to download a file from opentopography (I've removed the API key):
https://portal.opentopography.org/API/globaldem?demtype=COP30&south=40.95399473322263&north=42.593615267006946&west=69.65505930582663&east=71.75292435647813&outputFormat=GTiff
This might take a little while, depending on the size of the file. 
The filename will be:
./RC_COP30.tif
The path and file without path are:
./  RC_COP30.tif
Finished downloading
You are converting a D

'RC_COP30_UTM.bil'

## Now process the data

Now lets use *lsdtopotools* to extact channels and drainage areas. We first need to import the `lsdmapwrappers` module, and then run the code.

In [16]:
import lsdviztools.lsdmapwrappers as lsdmw

Dataset_prefix = "RC"
source_name = "COP30"

## Get the basins
lsdtt_parameters = {"write_hillshade" : "true",
                    "print_channels_to_csv" : "true",
                    "carve_before_fill" : "true",
                    "print_d8_drainage_area_raster" : "true",
                    "threshold_contributing_pixels" : "10000"}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

The lsdtopotools command line tools available are: 
['lsdtt-basic-metrics', 'lsdtt-channel-extraction', 'lsdtt-chi-mapping', 'lsdtt-cosmo-tool', 'lsdtt-hillslope-channel-coupling', 'lsdtt-valley-metrics']
Testing has been done against lsdtopotools v0.7
The command line tool is: lsdtt-basic-metrics
The driver name is: Test_01
The read path is: ./
The write path is: ./
The read prefix is: RC_COP30_UTM
The write prefix is: RC_COP30_UTM
The parameter dictionary is:
{'write_hillshade': 'true', 'print_channels_to_csv': 'true', 'carve_before_fill': 'true', 'print_d8_drainage_area_raster': 'true', 'threshold_contributing_pixels': '10000'}
Done writing the driver file
I've finised writing the driver file. Let me run LSDTT for you.


Now we would like to "burn" the drainage area data onto the channel file.

**Important: this routine does not check for completeness of the basins, so drainage area will be incorrect for basins draining from the edge**

In [19]:
import lsdviztools.lsdmapwrappers as lsdmw

burn_raster_prefix = Dataset_prefix+"_"+source_name +"_UTM_d8_area"
csv_name = Dataset_prefix+"_"+source_name +"_UTM_CN.csv"

## Get the basins
lsdtt_parameters = {"burn_raster_to_csv" : "true",
                    "burn_raster_prefix" : burn_raster_prefix,
                    "burn_data_csv_column_header" : "drainage_area",
                    "csv_to_burn_name" : csv_name}
r_prefix = Dataset_prefix+"_"+source_name +"_UTM"
w_prefix = Dataset_prefix+"_"+source_name +"_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-basic-metrics",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

The lsdtopotools command line tools available are: 
['lsdtt-basic-metrics', 'lsdtt-channel-extraction', 'lsdtt-chi-mapping', 'lsdtt-cosmo-tool', 'lsdtt-hillslope-channel-coupling', 'lsdtt-valley-metrics']
Testing has been done against lsdtopotools v0.7
The command line tool is: lsdtt-basic-metrics
The driver name is: Test_01
The read path is: ./
The write path is: ./
The read prefix is: RC_COP30_UTM
The write prefix is: RC_COP30_UTM
The parameter dictionary is:
{'burn_raster_to_csv': 'true', 'burn_raster_prefix': 'RC_COP30_UTM_d8_area', 'burn_data_csv_column_header': 'drainage_area', 'csv_to_burn_name': 'RC_COP30_UTM_CN.csv'}
Done writing the driver file
I've finised writing the driver file. Let me run LSDTT for you.


## Have a look at the channel data

In [8]:
import pandas as pd

burned_csv_name = Dataset_prefix+"_"+source_name +"_UTM_burned.csv"
df = pd.read_csv(burned_csv_name)
df.head()

,latitude,longitude,JunctionIndex,NI,StreamOrder,drainage_area,receiver_JI,receiver_NI
0,42.593753,71.620026,0,1905,1,954900.0,0,1905
1,42.593571,71.568466,1,4075,1,906300.0,2,4076
2,42.593563,71.568832,1,4076,1,927000.0,2,4077
3,42.593554,71.569199,2,4077,1,1055700.0,2,4077
4,42.592945,71.596230,3,4151,1,1534500.0,4,4152


## Get the basemaps and the channel images

In [9]:
import lsdviztools.lsdbasemaptools as bmt

In [51]:
DataDir = "./"
RasterF = Dataset_prefix+"_"+source_name +"_UTM.bil"

bmt.GenerateBasemapImageOrthographic(DataDir, RasterF, out_fname_prefix = "ortho_basemap",FigWidthInches=2,FigHeightInches=2,FigFormat="jpeg")


Trying to create a shapefile.
The Data directory is: ./ and the raster is: RC_COP30_UTM.bil
Yoyoyoyo the EPSG is :EPSG:32642
EPSG:32642
The raster has coordinate of: EPSG:32642
This ESPG is: 32642
I need to do something completely stupid since OGR made a terrible decision.


In [11]:
bmt.GenerateBasemapImageHillshade(DataDir, RasterF, "my_OT_api_key.txt",out_fname_prefix = "hillshade_basemap", FigWidthInches=2,FigFormat="jpeg")

Trying to create a shapefile.
The Data directory is: ./ and the raster is: RC_COP30_UTM.bil
Yoyoyoyo the EPSG is :EPSG:32642
EPSG:32642
The raster has coordinate of: EPSG:32642
This ESPG is: 32642
I need to do something completely stupid since OGR made a terrible decision.
The long dimension is: 185520.0
The aspect ratio is: 0.9592496765847348
Extents are: 
[54.351754726743465, 87.1104215506813, 26.061180373845193, 57.48492093005464]
I am taking your coordinates from the lower left list
I am taking your coordinates from the upper right list
I am reading you OT API key from the file my_OT_api_key.txt
Your source has 500 m grid spacing
The grid spacing for your DEM will be:500
I am going to download a file from opentopography (I've removed the API key):
https://portal.opentopography.org/API/globaldem?demtype=SRTM15Plus&south=26.061180373845193&north=57.48492093005464&west=54.351754726743465&east=87.1104215506813&outputFormat=GTiff
This might take a little while, depending on the size of 

/usr/local/lib/python3.10/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/50m_physical/ne_50m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


Now for the channel network image

In [44]:

from lsdviztools.lsdplottingtools import lsdmap_pointtools as LSDP
from lsdviztools.lsdmapfigure.plottingraster import MapFigure


burned_csv_name = Dataset_prefix+"_"+source_name +"_UTM_burned.csv"
hillshade_fname = Dataset_prefix+"_"+source_name +"_UTM_hs.bil"
DEM_fname = Dataset_prefix+"_"+source_name +"_UTM.bil"
cmap = "terrain"
thisPointData = LSDP.LSDMap_PointData(burned_csv_name)

# set up the base image and the map
print("I am getting the hillshade")

# If you want to turn off the background hillshade set alpha to 0
MF = MapFigure(hillshade_fname, "./",coord_type="None", colourbar_location="None", alpha = 1)
MF.add_drape_image(DEM_fname,"./",colourmap = cmap, alpha = 0.2)
MF.add_point_data(thisPointData,column_for_plotting = "None",
                  scale_points = True,
                  column_for_scaling = "drainage_area",
                  this_colourmap = "blues",
                  scaled_data_in_log = True,
                  max_point_size = 5,
                  min_point_size = 1)
MF.add_scalebar()

fig_name = MF.save_fig(fig_width_inches = 5, transparent=True, FigFileName = "awesome_network_image.jpeg",
                       Fig_dpi = 500, return_fig = False)

I did not find a valid separator. I am assuming the path is ./
The object file prefix is: RC_COP30_UTM_burned
Loading your file from csv
done
I am getting the hillshade
Your colourbar will be located: None
xsize: 5932 and y size: 6184
NoData is: -9999.0
Yoyoyoyo the EPSG is :EPSG:32642
EPSG:32642
The EPSGString is: EPSG:32642
I made the ticks.
x labels are: 
[]
x locations are:
[]
y labels are: 
[]
y locations are:
[]
This colourmap is: gray
The number of axes are: 1
Axes(0,0;1x1)
Axes(0,0;1x1)
N axes are: 1
Axes(0,0;1x1)
xsize: 5932 and y size: 6184
NoData is: -9999.0
Yoyoyoyo the EPSG is :EPSG:32642
EPSG:32642
The EPSGString is: EPSG:32642
I am going to use the normalisation None
I am using the full range of values in the raster.
The number of axes are: 2
I am going to plot some points for you. The EPSG string is:EPSG:32642
pointtools GetUTMEastingNorthing, getting the epsg string: EPSG:32642
WARNING you must have a recent (>=6) version of proj and pyproj (>=2.4) for this to work 
EP

/usr/local/lib/python3.10/dist-packages/lsdviztools/lsdmapfigure/plottingraster.py:1626: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  sc = self.ax_list[0].scatter(easting,northing,s=point_scale, c= unicolor,cmap=this_colourmap,edgecolors='none', alpha = alpha,zorder=zorder, marker = marker)


In [30]:
# Download the Roboto Condensed font
!wget -q https://github.com/googlefonts/roboto/blob/main/src/hinted/RobotoCondensed-Regular.ttf?raw=true -O RobotoCondensed-Regular.ttf

# Create a directory for the font and move the font file there
!mkdir -p ~/.fonts
!mv RobotoCondensed-Regular.ttf ~/.fonts/

# Refresh the font cache
!fc-cache -fv

# Verify the font is installed
!fc-list | grep "Roboto"

/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 2 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded
/root/.fonts/RobotoCondensed-Regular.ttf: Roboto Condensed:style=Regular


In [52]:
from PIL import Image, ImageDraw, ImageFont, ImageChops

def trim_whitespace(image):
    bg = Image.new(image.mode, image.size, image.getpixel((0, 0)))
    diff = ImageChops.difference(image, bg)
    bbox = diff.getbbox()
    if bbox:
        return image.crop(bbox)
    return image

# Load the images
img1 = Image.open("awesome_network_image.jpeg")
img2 = Image.open("ortho_basemap_basemap.jpeg")
img3 = Image.open("hillshade_basemap_basemap.jpeg")

# Trim whitespace around the images
img1 = trim_whitespace(img1)
img2 = trim_whitespace(img2)
img3 = trim_whitespace(img3)

# Get the dimensions of the images
width1, height1 = img1.size
width2, height2 = img2.size
width3, height3 = img3.size

# Define the buffers
outer_buffer = 30
column_buffer = 20
image_buffer = 200  # Updated to 100 px between img2 and img3
first_image_buffer = 100

# Calculate the width and height of the final image
final_width = width1 + column_buffer + max(width2, width3) + 2 * outer_buffer + first_image_buffer
final_height = max(height1 + 2 * first_image_buffer, height2 + height3 + image_buffer) + 2 * outer_buffer

# Create a new image with a white background
final_img = Image.new("RGB", (final_width, final_height), "white")

# Paste the first image in the first column with outer buffer and first image buffer
final_img.paste(img1, (outer_buffer + first_image_buffer, outer_buffer + first_image_buffer))

# Calculate the y position to paste the second and third images so they are centered vertically
second_column_height = height2 + height3 + image_buffer
y_offset = (final_height - second_column_height) // 2

# Calculate the x position to center the second and third images horizontally
x_offset = width1 + column_buffer + outer_buffer + first_image_buffer
x_centered2 = x_offset + (max(width2, width3) - width2) // 2
x_centered3 = x_offset + (max(width2, width3) - width3) // 2

# Paste the second image in the second column
final_img.paste(img2, (x_centered2, y_offset))

# Paste the third image below the second image in the second column
final_img.paste(img3, (x_centered3, y_offset + height2 + image_buffer))

# Add text to the bottom left
draw = ImageDraw.Draw(final_img)
font = ImageFont.truetype("/root/.fonts/RobotoCondensed-Regular.ttf", 48)  # Path to the font file

text = "Image by Simon M Mudd using lsdtopotools. Channel network to the east of Tashkent"
text_position = (outer_buffer, final_height - outer_buffer - 50)  # Adjust the position as needed

draw.text(text_position, text, fill="black", font=font)

# Save the final image
final_img.save("combined_image.jpg")

print("The images have been combined successfully and saved as combined_image.jpg.")


The images have been combined successfully and saved as combined_image.jpg.
